<a href="https://colab.research.google.com/github/Naved555/Spring_5731/blob/main/Jiwani_Naved_Exercise_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 In-class Exercise 4**

**This exercise will provide a valuable learning experience in working with text data and extracting features using various topic modeling algorithms. Key concepts such as Latent Dirichlet Allocation (LDA), Latent Semantic Analysis (LSA), lda2vec, and BERTopic.**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


***Please use the text corpus you collected in your last in-class-exercise for this exercise. Perform the following tasks***.

**Expectations**:
*   Students are expected to complete the exercise during lecture period to meet the active participation criteria of the course.
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

**Total points**: 40

**Deadline**: This in-class exercise is due at the end of the day tomorrow, at 11:59 PM.

**Late submissions will have a penalty of 10% of the marks for each day of late submission, and no requests will be answered. Manage your time accordingly.**


## Question 1 (10 Points)

**Generate K topics by using LDA, the number of topics K should be decided by the coherence score, then summarize what are the topics.**

You may refer the code here: https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [2]:
# Write your code here
import csv
import json
import nltk
import pandas as pd
nltk.download('stopwords')

with open('/root/cleaned_data.csv', 'r') as f:
    data = list(csv.reader(f))

df = pd.DataFrame(data)
print(df.head())



    0                      1                    2  \
0  ID                   Name             Birthday   
1   0           Kay Aiko Abe  1927-05-09T00:00:00   
2   1                Art Abe  1921-06-12T00:00:00   
3   2  Sharon Tanagi Aburano  1925-10-31T00:00:00   
4   3        Toshiko Aiboshi  1928-04-08T00:00:00   

                                                   3  \
0                                                Bio   
1  Nisei female. Born May 9, 1927, in Selleck, Wa...   
2  Nisei male. Born June 12, 1921, in Seattle, Wa...   
3  Nisei female. Born October 31, 1925, in Seattl...   
4  Nisei female. Born July 8, 1928, in Boyle Heig...   

                                                   4  
0                                        Cleaned_Bio  
1  nisei femal born may 9 1927 selleck washington...  
2  nisei male born june 12 1921 seattl washington...  
3  nisei femal born octob 31 1925 seattl washingt...  
4  nisei femal born juli 8 1928 boyl height calif...  


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
!pip install pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 35.3 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.2.1 which is incompatible.


In [4]:
!pip install pandas==1.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 49.6 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.1
    Uninstalling pandas-2.2.1:
      Successfully uninstalled pandas-2.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyldavis 3.4.1 requires pandas>=2.0.0, but you have pandas 1.5.3 which is incompatible.


In [26]:
# Importing necessary libraries
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
from gensim.models import LdaMulticore
from gensim.corpora import Dictionary
from gensim.models import CoherenceModel
import re
from gensim.utils import simple_preprocess
import nltk
from nltk.corpus import stopwords
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
import matplotlib.pyplot as plt

# Read data into nav DataFrame
nav = pd.read_csv('/root/cleaned_data.csv')
print(nav.columns)

# Preprocess text data
nav['processed_text'] = nav['Name'].map(lambda x: re.sub('[,!?]', '', x))
nav['processed_text'] = nav['processed_text'].map(lambda x: x.lower())

# Tokenize and remove stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')


def tokenize(text):
    return [word for word in simple_preprocess(text) if word not in stop_words]

nav['tokenized_text'] = nav['processed_text'].map(tokenize)

# Create a dictionary and corpus
id2word = Dictionary(nav['tokenized_text'])
corpus = [id2word.doc2bow(text) for text in nav['tokenized_text']]

# Determine the optimal number of topics using coherence score
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=1):
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = LdaMulticore(corpus=corpus, id2word=dictionary, num_topics=num_topics, workers=1)  # Set workers to 1
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

# Compute coherence scores
start_topic = 2  # Specify the starting number of topics
limit_topic = 20  # Specify the upper bound for the number of topics
step_topic = 1  # Specify step size
model_list, coherence_values = compute_coherence_values(dictionary=id2word,
                                                        corpus=corpus,
                                                        texts=nav['tokenized_text'],
                                                        limit=limit_topic,
                                                        start=start_topic,
                                                        step=step_topic)


# Find the optimal number of topics based on coherence score
optimal_num_topics = start_topic + coherence_values.index(max(coherence_values))

# Build LDA model with optimal number of topics
lda_model = LdaMulticore(corpus=corpus, id2word=id2word, num_topics=optimal_num_topics, workers=1)  # Set workers to 1

# Print the topics
topics = lda_model.print_topics()
for topic in topics:
    print(topic)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Index(['ID', 'Name', 'Birthday', 'Bio', 'Cleaned_Bio'], dtype='object')


(0, '0.014*"george" + 0.007*"watanabe" + 0.006*"mary" + 0.005*"frank" + 0.005*"murakami" + 0.004*"kay" + 0.004*"sato" + 0.004*"roy" + 0.004*"john" + 0.004*"bill"')
(1, '0.007*"mary" + 0.006*"yamamoto" + 0.005*"frank" + 0.004*"nakano" + 0.004*"bill" + 0.004*"george" + 0.004*"kenji" + 0.004*"james" + 0.003*"jim" + 0.003*"ito"')


## Question 2 (10 Points)

**Generate K topics by using LSA, the number of topics K should be decided by the coherence score, then summarize what are the topics.**

You may refer the code here: https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python

In [28]:
# Write your code here
import pandas as pd
from gensim.models import LsiModel
from gensim.models.coherencemodel import CoherenceModel
from gensim.parsing.preprocessing import remove_stopwords, strip_punctuation, strip_short, stem_text, preprocess_string
from gensim import corpora
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Assuming nav is your DataFrame containing Name_Info column

# Preprocess text data
def preprocess(text):
    CUSTOM_FILTERS = [lambda x: x.lower(),
                      remove_stopwords,
                      strip_punctuation,
                      strip_short,
                      stem_text]
    text = preprocess_string(text, CUSTOM_FILTERS)
    return text

print(nav.columns)  # Corrected typo
# Preprocess text data
nav['Text (Clean)'] = nav['Name'].apply(lambda x: preprocess(x))  # Corrected DataFrame name

# Create a dictionary with the corpus
dictionary = corpora.Dictionary(nav['Text (Clean)'])  # Corrected variable name

# Convert corpus into a bag of words
bow = [dictionary.doc2bow(text) for text in nav['Text (Clean)']]  # Corrected variable name

# Find the optimal number of topics using coherence score
coherence_values = []
for num_topics in range(2, 11):
    lsi = LsiModel(bow, num_topics=num_topics, id2word=dictionary)
    coherence_model = CoherenceModel(model=lsi, texts=nav['Text (Clean)'], dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    coherence_values.append((num_topics, coherence_score))
    print('Coherence score with {} clusters: {}'.format(num_topics, coherence_score))

# Choose the optimal number of topics based on coherence score
optimal_num_topics = max(coherence_values, key=lambda x: x[1])[0]

# Perform LSA to generate K topics
lsi = LsiModel(bow, num_topics=optimal_num_topics, id2word=dictionary)


Index(['ID', 'Name', 'Birthday', 'Bio', 'Cleaned_Bio', 'processed_text',
       'tokenized_text', 'Text (Clean)'],
      dtype='object')
Coherence score with 2 clusters: 0.7218997496386413
Coherence score with 3 clusters: 0.7253380359053887
Coherence score with 4 clusters: 0.6950874964484917
Coherence score with 5 clusters: 0.7169365898272845
Coherence score with 6 clusters: 0.6910716793598647
Coherence score with 7 clusters: 0.6786220812614864
Coherence score with 8 clusters: 0.7285658202914547
Coherence score with 9 clusters: 0.7174757084604688
Coherence score with 10 clusters: 0.677427421188213


In [31]:
# Print the top words in each topic
for topic_num, words in lsi.print_topics(num_words=10):  # Updated variable name in the loop
    print('Words in topic {}: {}.'.format(topic_num, words))

# Find the scores given between the review and each topic
corpus_lsi = lsi[bow]
topic_scores = []
for doc in corpus_lsi:
    topic_scores.append([round(val[1], 2) for val in doc])


Words in topic 0: -0.978*"georg" + -0.048*"murakami" + -0.046*"joe" + -0.044*"nakano" + -0.041*"kiyo" + -0.040*"yamada" + -0.040*"yoshida" + -0.039*"matsumoto" + -0.038*"maeda" + -0.038*"iseri".
Words in topic 1: 0.951*"mari" + 0.145*"watanab" + 0.059*"jane" + 0.056*"suzuki" + 0.056*"nakamura" + 0.054*"okazaki" + 0.054*"ikeda" + 0.053*"yoshida" + 0.053*"nakata" + 0.053*"tomita".
Words in topic 2: 0.956*"frank" + 0.124*"sato" + 0.075*"emi" + 0.074*"fujii" + 0.071*"fukuhara" + 0.064*"sumida" + 0.063*"konishi" + 0.061*"sata" + 0.060*"kikuchi" + 0.060*"yamasaki".
Words in topic 3: 0.940*"watanab" + -0.141*"mari" + 0.124*"grace" + 0.080*"ann" + 0.080*"june" + 0.078*"kimura" + 0.069*"martha" + 0.068*"yuki" + 0.067*"amatatsu" + 0.066*"iku".
Words in topic 4: 0.930*"yamamoto" + 0.196*"richard" + 0.113*"murakami" + 0.088*"sumiko" + 0.084*"emi" + 0.080*"kazu" + 0.079*"yuriko" + 0.078*"kaz" + 0.070*"madelon" + 0.070*"arai".
Words in topic 5: 0.751*"murakami" + 0.483*"richard" + 0.219*"roi" + -0.1

In [32]:
# Create a DataFrame to show scores assigned for each topic for each review
topic_df = pd.DataFrame(topic_scores, columns=['Topic {}'.format(i) for i in range(optimal_num_topics)])  # Updated variable name
print(topic_df)  # Corrected line
topic_df['Text'] = nav['Text (Clean)']  # Corrected DataFrame name
topic_df['Dominant Topic'] = topic_df.iloc[:, :optimal_num_topics].idxmax(axis=1)

# Find a sample review from each topic
for i in range(optimal_num_topics):
    topic_df_i = topic_df[topic_df['Dominant Topic'] == 'Topic {}'.format(i)]  # Updated variable name
    if not topic_df_i.empty:
        sample_text = topic_df_i.sample(1, random_state=2)['Text'].values[0]
        print('Sample text from topic {}:\n{}'.format(i, sample_text))
    else:
        print('No sample text available for topic {}'.format(i))


     Topic 0  Topic 1  Topic 2  Topic 3  Topic 4  Topic 5  Topic 6  Topic 7
0       -0.0      0.0    -0.01     0.01     0.01    -0.01     0.02    -0.06
1        0.0     -0.0     0.01    -0.00     0.01    -0.00     0.00     0.00
2       -0.0      0.0    -0.00     0.00    -0.00    -0.02    -0.01    -0.01
3        0.0      0.0     0.00    -0.01    -0.01     0.02     0.00     0.00
4        0.0     -0.0    -0.00    -0.00     0.00     0.00     0.00    -0.00
..       ...      ...      ...      ...      ...      ...      ...      ...
972      0.0      0.0    -0.00    -0.00     0.00    -0.00     0.00    -0.00
973     -0.0      0.0     0.00    -0.00     0.00     0.02     0.92     0.01
974     -0.0     -0.0     0.00     0.00    -0.00    -0.01    -0.01    -0.01
975      0.0      0.0    -0.00     0.07    -0.00     0.01    -0.01    -0.01
976      0.0      0.0    -0.00    -0.00     0.00     0.01    -0.00     0.00

[977 rows x 8 columns]
Sample text from topic 0:
['toyoko', 'okumura']
Sample text from

## Question 3 (10 points):
**Generate K topics by using lda2vec, the number of topics K should be decided by the coherence score, then summarize what are the topics.**

You may refer the code here: https://nbviewer.org/github/cemoody/lda2vec/blob/master/examples/twenty_newsgroups/lda2vec/lda2vec.ipynb

In [19]:
!pip install bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 65.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 20.8 MB/s eta 0:00:00
  Using cached Cython-0.29.37-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 56.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/73

In [38]:
# Write your code here
import pandas as pd
from bertopic import BERTopic
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary

# Read the CSV file
data = pd.read_csv("/root/cleaned_data.csv")  # Changed variable name to 'data'

# Initialize BERTopic model
topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True)

# Convert values in 'Name_Info' column to strings
data['Name_Info'] = data['Name'].astype(str)  # Updated column name

# Fit BERTopic model on the preprocessed text data
topics, probabilities = topic_model.fit_transform(data['Name_Info'])

# Get top words per topic
topics_info = topic_model.get_topics()
top_words_per_topic = [[word for word, _ in words] for topic_id, words in topics_info.items() if topic_id != -1]

# Tokenize and preprocess documents
tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words('english'))
processed_docs = [[word for word in tokenizer.tokenize(doc.lower()) if word not in stop_words] for doc in data['Name_Info'].astype(str).tolist()]

# Create dictionary and corpus for coherence calculation
dictionary = Dictionary(processed_docs)
corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

# Calculate coherence score
coherence_model = CoherenceModel(topics=top_words_per_topic, texts=processed_docs, dictionary=dictionary, coherence='c_v')
coherence_score = coherence_model.get_coherence()
print(f"Coherence Score: {coherence_score}")

# Determine the number of topics
num_topics = len(topics_info) - 1 if -1 in topics_info else len(topics_info)
print(f"Number of Topics (K): {num_topics}")

# Summarize Topics
print("\nTopic Summaries:")
for topic_id, words in topics_info.items():
    if topic_id != -1:
        topic_summary = ", ".join([word for word, _ in words])
        print(f"Topic {topic_id}: {topic_summary}\n")
    else:
        break


2024-03-30 01:19:13,857 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/31 [00:00<?, ?it/s]

2024-03-30 01:19:19,581 - BERTopic - Embedding - Completed ✓
2024-03-30 01:19:19,584 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-03-30 01:19:23,927 - BERTopic - Dimensionality - Completed ✓
2024-03-30 01:19:23,930 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-03-30 01:19:23,999 - BERTopic - Cluster - Completed ✓
2024-03-30 01:19:24,006 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-03-30 01:19:24,024 - BERTopic - Representation - Completed ✓


Coherence Score: 0.5195231332289313
Number of Topics (K): 2

Topic Summaries:
Topic 0: george, mary, frank, watanabe, bill, murakami, john, james, sato, kay

Topic 1: yamamoto, eric, hisaye, arai, kojima, madelon, fusako, takenori, tats, yoneo



## Question 4 (10 points):
**Generate K topics by using BERTopic, the number of topics K should be decided by the coherence score, then summarize what are the topics.**

You may refer the code here: https://colab.research.google.com/drive/1FieRA9fLdkQEGDIMYl0I3MCjSUKVF8C-?usp=sharing

In [46]:
# Write your code here
from bertopic import BERTopic
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary

# Initialize BERTopic model
topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True)

# Assuming nav is your DataFrame containing the data
# Tokenize and preprocess documents
tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words('english'))

# Fit BERTopic model on the preprocessed text data
topics, probabilities = topic_model.fit_transform(nav['processed_text'].tolist())  # Assuming 'processed_text' contains preprocessed text data

# Get top words per topic
topics_info = topic_model.get_topics()
top_words_per_topic = [[word for word, _ in words] for topic_id, words in topics_info.items() if topic_id != -1]

# Tokenize each document separately
tokenized_docs = [doc.split() for doc in nav['processed_text']]
# Create dictionary and corpus for coherence calculation
dictionary = Dictionary(tokenized_docs)
corpus = [dictionary.doc2bow(doc) for doc in tokenized_docs]

# Calculate coherence score
coherence_model = CoherenceModel(topics=top_words_per_topic, texts=tokenized_docs, dictionary=dictionary, coherence='c_v')
coherence_score = coherence_model.get_coherence()
print(f"Coherence Score: {coherence_score}")

# Determine the number of topics
num_topics = len(topics_info) - 1 if -1 in topics_info else len(topics_info)
print(f"Number of Topics (K): {num_topics}")

# Summarize Topics
print("\nTopic Summaries:")
for topic_id, words in topics_info.items():
    if topic_id != -1:
        topic_summary = ", ".join([word for word, _ in words])
        print(f"Topic {topic_id}: {topic_summary}\n")
    else:
        break  # No more topics to print


2024-03-30 01:28:29,432 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/31 [00:00<?, ?it/s]

2024-03-30 01:28:34,508 - BERTopic - Embedding - Completed ✓
2024-03-30 01:28:34,511 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-03-30 01:28:38,918 - BERTopic - Dimensionality - Completed ✓
2024-03-30 01:28:38,920 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-03-30 01:28:38,990 - BERTopic - Cluster - Completed ✓
2024-03-30 01:28:38,996 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-03-30 01:28:39,014 - BERTopic - Representation - Completed ✓


Coherence Score: 0.5188527786198915
Number of Topics (K): 2

Topic Summaries:
Topic 0: george, mary, frank, watanabe, bill, murakami, john, james, sato, kay

Topic 1: yamamoto, eric, hisaye, arai, kojima, madelon, fusako, takenori, tats, yoneo



## Extra Question (5 Points)

**Compare the results generated by the four topic modeling algorithms, which one is better? You should explain the reasons in details.**

**This question will compensate for any points deducted in this exercise. Maximum marks for the exercise is 40 points.**

In [ ]:
# Write your code here
'''
LDA:The result shows terms and their probabilities, showing whether or not they are associated with sentiment analysis
    positively or negatively. positive associations are generally associated with pleasant emotions like contentment
    or happiness, whereas negative associations are associated with negative emotions like sadness or discontent.
LSA:takes sample text from each topic and uses coherence scores to determine if the material expresses a favorable
     or negative sentiment. These ratings offer a measure of the issues' coherence or consistency, which may be a proxy for
     the emotions expressed in them.
lda2vec and BERTopic:Each topic is summarized in the output by emphasizing important terms related to the subject.
                     This succinct synopsis makes it easier to comprehend the key ideas contained in each topic, which
                     facilitates interpretation and analysis.
Generally, LDA and LSA perform better than BERTopic and lda2vec. In situations where clear insights are crucial,
LDA in particular is preferred due to its simplicity and interpretability. On the other hand, even if they may execute more slowly
or with greater complexity, lda2vec and BERTopic are better suited to extract more complicated associations from the data.

'''

# Mandatory Question

**Important: Reflective Feedback on this exercise**

Please provide your thoughts and feedback on the exercises you completed in this assignment.

Consider the following points in your response:

**Learning Experience:** Describe your overall learning experience in working with text data and extracting features using various topic modeling algorithms. Did you understand these algorithms and did the implementations helped in grasping the nuances of feature extraction from text data.

**Challenges Encountered:** Were there specific difficulties in completing this exercise?

Relevance to Your Field of Study: How does this exercise relate to the field of NLP?

**(Your submission will not be graded if this question is left unanswered)**



In [ ]:
# Your answer here (no code for this question, write down your answer as detail as possible for the above questions):

'''
Please write you answer here:
It gave exposure to various topic modeling algorithms such as Latent Dirichlet Allocation (LDA), Latent Semantic Analysis (LSA), lda2vec, and BERTopic.
and understanding their differences helps in choosing the appropriate one for different tasks. It was my first time but a fun valuable learning. the challenge i faced in 3rd question was i was unable to
install the lda2vec in the google colab even after multiple attempt so then i continued with BERTopic everything else went well took some time for errors but able to execute with the help of online tools.
'''